In [ ]:
import aiida

# must execute it in the first place
aiida.load_profile()


In [ ]:
from aiida.orm import Code
from aiida.orm import Str, Int, Dict, List, Float
from aiida.engine import calcfunction, WorkChain, ToContext, append_
from aiida.plugins import DataFactory, WorkflowFactory
from itertools import cycle
import os
from tools import wait_for_node_finished

from aiida.engine import calcfunction, workfunction, submit, run
from aiida.orm import load_code, load_node

# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')


In [ ]:
# codeの定義を行う。
from tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os

home = expanduser("~")
work_directory = os.path.join(home, 'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')
print(computer_local)
code = get_or_create_code('alamode.displace_pf',
                          computer_local,
                          '/home/max/Documents/aiida-test/alamode-aiida/alamode/displace.py')
code_displace_pf = code
code = get_or_create_code('alamode.displace_random',
                          computer_local,
                          '/home/max/Documents/aiida-test/alamode-aiida/alamode/displace.py')
code_displace_random = code
ode = get_or_create_code('alamode.lammps',
                         computer_local,
                         '/usr/bin/lammps')
code_lammps_name = Str('alamode.lammps@localhost')

code_displace_random, code_displace_pf, code_lammps_name


In [ ]:
import json
with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path


In [ ]:
from os.path import expanduser
from tools import NodeBank


# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)


In [ ]:
g_cwd = nodebank.load("cubic_cwd")  # cubic term 保存directory
g_prefix = nodebank.load('cubic_prefix')  # cubic term prefix
g_dispfile_folder = nodebank.load(
    "suggest_cubic_pattern_folder")  # alm suggestの結果
g_displace_result = nodebank.load("suggest_cubic_result")  # alm suggestの結果
g_superstructure = nodebank.load("superstructure")  # 長周期構造


In [ ]:
g_prefix, g_cwd, g_displace_result, g_dispfile_folder


# displace

In [ ]:
g_format = nodebank.load("format")  # structure file format
g_superstructure_filepath = nodebank.load(
    "superstructure_filepath")  # structurefile path
g_cubic_mag = nodebank.load_or_dump("cubic_mag", Float(0.04))
g_format, g_cubic_mag, g_superstructure_filepath


In [ ]:
g_alm_suggest_cubic = nodebank.load("suggest_cubic_result", raise_error=True)
g_cubic_pattern_files = List(
    list=[g_alm_suggest_cubic["disp_pattern"]['ANHARM3']])
g_cubic_pattern_files


In [ ]:
g_action = "pf"

if g_action == "pf":
    #codename = "alamode.displace_pf@tutor"
    #code = Code.get_from_string(codename)
    code = code_displace_pf

    builder = code.get_builder()
    builder.format = g_format
    builder.cwd = g_cwd
    builder.structure_org = g_superstructure_filepath
    builder.mag = g_cubic_mag
    builder.prefix = g_prefix
    builder.pattern_files = g_cubic_pattern_files

    g_displacefuture = nodebank.load("displace_cubic_pf", g_force)
    if g_displacefuture is None:
        g_displacefuture = submit(builder)
        print(g_displacefuture)
        wait_for_node_finished(g_displacefuture)
        if g_displacefuture.is_finished_ok:
            nodebank.dump("displace_cubic_pf", g_displacefuture)

elif g_action == "random":
    #codename = "alamode.displace_random@tutor"
    #code = Code.get_from_string(codename)
    code = code_displace_random

    builder = code.get_builder()
    builder.format = g_format
    builder.cwd = g_cwd
    builder.prefix = g_prefix
    builder.structure_org = g_superstructure_filepath
    builder.mag = g_cubic_mag
    builder.num_disp = g_num_disp

    g_displacefuture = nodebank.load("displace_cubic_random", g_force)
    if g_displacefuture is None:
        g_displacefuture = submit(builder)
        print(g_displacefuture)
        wait_for_node_finished(g_displacefuture)
        if g_displacefuture.is_finished_ok:
            nodebank.dump("displace_cubic_random", g_displacefuture)
        else:
            raise
else:
    raise ValueError("unknown action")


In [ ]:
g_displacefuture.outputs.dispfile_folder.list_object_names()


In [ ]:
_dir = os.path.abspath('lammps_input/Si222')
if True:
    lammps_potentials = FolderData()
    for _p in ["Si.sw"]:
        abspath = os.path.join(_dir, _p)
        print(abspath, _p)
        lammps_potentials.put_object_from_file(abspath, path=_p)
else:
    _potential = []
    for _p in ["Si.sw"]:
        _potential.append(os.path.join(_dir, _p))
    lammps_potentials = List(list=_potential)

input_file_template = SinglefileData(os.path.join(_dir, "in.sw"))


In [ ]:
g_dispfile_folder = g_displacefuture.outputs.dispfile_folder
g_dispfile_folder, g_dispfile_folder.list_object_names()


In [ ]:
g_displace_result = g_displacefuture.outputs.result
g_displace_result.attributes


In [ ]:
codename = code_lammps_name


In [ ]:
inputs = {"displace_result": g_displace_result,
          "data_folder": g_dispfile_folder,
          "input_file_template":  input_file_template,
          "code_string":  codename,
          'prefix': g_prefix,
          "cwd": g_cwd,
          'potential_files': lammps_potentials}


In [ ]:
alldisp_lammps_WorkChain = WorkflowFactory("alamode.dispall_lammps")


In [ ]:
lammps_all = nodebank.load("all_lammps_harmonic")
if lammps_all is None:
    lammps_all = submit(alldisp_lammps_WorkChain, **inputs)
    wait_for_node_finished(lammps_all, 5)
    print(lammps_all.is_finished_ok)


In [ ]:
lammps_all

In [ ]:
lammps_all.outputs.dfset

In [ ]:
nodebank.dump("cubic_all_dfset",lammps_all.outputs.dfset)


In [ ]:
"done"